Date: 2016-10-06

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
%qtconsole
import os
import sys
import collections
import random
import itertools
import functools
import scipy.fftpack
import scipy.signal
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as clr
import matplotlib.patches as patches
import seaborn as sns
import pandas as pd
import tqdm
import dask
import dask.multiprocessing

sys.path.append('../src/')
import data_filter
import ripples
import spectral

In [ ]:
Animal = collections.namedtuple('Animal', {'directory', 'short_name'})
num_days = 8
days = range(1, num_days + 1)
animals = {'HPa': Animal(directory='HPa_direct', short_name='HPa'),
           'HPc': Animal(directory='HPc_direct', short_name='HPc')}

epoch_info = data_filter.make_epochs_dataframe(animals, days)
tetrode_info = data_filter.make_tetrode_dataframe(animals)
epoch_keys = data_filter.get_dataframe_index(epoch_info
    .loc[(['HPa', 'HPc'], [8]), :]
    .loc[epoch_info.environment == 'wtr1'])

cur_tetrode_info = [tetrode_info[key][tetrode_info[key].area.isin(['PFC', 'CA1'])]
                    for key in epoch_keys]
print(cur_tetrode_info)

tetrode_index = [data_filter.get_dataframe_index(info) for info in cur_tetrode_info]
lfp_data = [data_filter.get_LFP_data(index, animals) for index in tetrode_index]
ripple_times = [ripples._get_computed_consensus_ripple_times(key, animals) for key in epoch_keys]

In [ ]:
params = dict(
    baseline_window=(-0.250, 0),
    window_of_interest=(-0.250, .250),
    sampling_frequency=1500,
    time_window_duration=0.040,
    time_window_step=0.010,
    desired_frequencies=[100, 300],
    time_halfbandwidth_product=1
)
    
coh = [[spectral.difference_from_baseline_coherence([lfp1, lfp2], epoch_ripple_times, **params)
        for lfp1, lfp2 in itertools.combinations(epoch_lfps, 2)]
       for epoch_lfps, epoch_ripple_times
       in zip(lfp_data, ripple_times)]

## Ripple Times Spectrum

In [ ]:
params = dict(
    sampling_frequency=1500,
    time_halfbandwidth_product=12,
)

def convert_ripple_times_to_dataframe(ripple_times, index):
    return (pd.DataFrame({'time': np.array(ripple_times)[:, 0], 'is_ripple': 1})
              .set_index('time')
              .reindex(index=index.values,
                       tolerance=1E-4,
                       method='nearest',
                       fill_value=0))

ripple_psd = [spectral.multitaper_power_spectral_density(
                convert_ripple_times_to_dataframe(epoch_ripple_times, epoch_lfps[0].index), **params)
              for epoch_lfps, epoch_ripple_times
              in zip(lfp_data, ripple_times)]

print(ripple_psd)

In [ ]:
ripple_psd[0].plot()
ripple_psd[1].plot()